In [616]:
import torch
import time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("当前设备名称：", torch.cuda.get_device_name(device) if device.type == 'cuda' else 'CPU')

当前设备名称： NVIDIA GeForce RTX 3080


> 1、生成词索引构成的序列

In [617]:
batch_size = 2
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5
# src_len = torch.randint(2,5,(batch_size,))
# tgt_len = torch.randint(2,5,(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)
# 词索引构成的序列
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len-L)), 0) \
                     for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len-L)), 0) \
                     for L in tgt_len])
print(src_seq)
print(tgt_seq)

tensor([[5, 1, 0, 0, 0],
        [3, 1, 7, 7, 0]])
tensor([[2, 1, 7, 6, 0],
        [7, 5, 6, 0, 0]])


> 2、采用Embedding的forward方法

In [618]:
# 实例化embedding # requires_grad 
# weight是随机的，训练时更新
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
# print(src_embedding_table.weight, tgt_embedding_table.weight)
# Embedding存在 --call-- forward方法隐式
src_embedding = src_embedding_table(src_seq)
tgt_embedding_table = tgt_embedding_table(tgt_seq)
print(src_embedding)
# 由于为0 底下3排一样

tensor([[[-0.2837, -0.4433,  0.6236,  0.1594,  1.9104, -0.5462,  1.0532,
          -0.0240],
         [-1.7059, -1.0046, -0.0326, -0.0299,  2.3425, -1.7953, -1.8357,
           0.7659],
         [-0.1718, -1.8654, -0.1987, -0.8014, -1.0334,  0.4024,  1.1313,
           1.7135],
         [-0.1718, -1.8654, -0.1987, -0.8014, -1.0334,  0.4024,  1.1313,
           1.7135],
         [-0.1718, -1.8654, -0.1987, -0.8014, -1.0334,  0.4024,  1.1313,
           1.7135]],

        [[-1.6621,  0.1629, -1.6330, -1.4075,  0.8662,  0.4910, -0.4970,
          -0.8949],
         [-1.7059, -1.0046, -0.0326, -0.0299,  2.3425, -1.7953, -1.8357,
           0.7659],
         [-0.3840, -0.3035,  0.7473, -0.4693,  0.8002, -0.2641, -1.3366,
           1.0725],
         [-0.3840, -0.3035,  0.7473, -0.4693,  0.8002, -0.2641, -1.3366,
           1.0725],
         [-0.1718, -1.8654, -0.1987, -0.8014, -1.0334,  0.4024,  1.1313,
           1.7135]]], grad_fn=<EmbeddingBackward0>)


> 3、生成pos—embedding

In [619]:
# pos embedding    method1 
start_time = time.time()
pos_embedding = torch.zeros(max_position_len, model_dim)
for i in range(0, model_dim):
    for j in range(0, max_position_len):
        if i%2 == 0:
            # print(i,j)
            pos_embedding[j, i] = np.sin(j/np.power(10000, (i/model_dim)))
        else:
            pos_embedding[j, i] = np.cos(j/np.power(10000, (i/model_dim)))
# print(pos_embedding)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("np Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# pos embedding    method2 
start_time = time.time()
pos_mat = torch.arange(max_position_len).reshape(-1, 1) 
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1))/model_dim)
i1_mat = torch.pow(10000, torch.arange(1, 8, 2).reshape((1, -1))/model_dim)
# print(pos_mat, i_mat, i1_mat)
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat/i_mat)
pe_embedding_table[:, 1::2] = torch.cos(pos_mat/i1_mat)
print(pe_embedding_table)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("torch Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# print(torch.sub(pos_embedding, pe_embedding_table))
pe_embedding = nn.Embedding(max_position_len, model_dim)
# 位置编码训练中不发生变化requires_grad=False
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
# 注意 这里给embedding的是位置 不是词的编号
# src_p=torch.zeros_like(src_seq)
# for i in range(src_p.size(0)):
#     for j in range(src_p.size(1)):
#         src_p[i][j] = j 
# print(src_p)
# src_pe_embedding =pe_embedding(src_p)
src_pe_embedding =pe_embedding(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
print(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
print(src_pe_embedding)
tgt_pe_embedding = src_pe_embedding

np Time (microseconds): 1001.1196136474609
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9995e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01,  3.0114e-01,  3.8942e-01,  9.9201e-01,  3.9989e-02,
          9.9992e-01,  4.0000e-03,  1.0000e+00]])
torch Time (microseconds): 997.5433349609375
tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
           9.9999e-01,  1.0000e-03,  1.0000e+

> 4、生成enc_self_attn_mask 无因果关系   对词向量相似度进行masked

In [620]:
# 这里不是三角矩阵# 这里不是三角矩阵 没有因果关系 多用矩阵相乘 转置 升维
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 2)
# print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# vaild_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len])
print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# 这里不是三角矩阵
# print(torch.tril(torch.ones(4, 4)))
# vaild_mask = vaild_encoder_pos.transpose(1,2)*torch.ones(4, 4)*vaild_encoder_pos
# print(vaild_mask.shape, "\n", vaild_mask)
# print(torch.bmm(torch.unsqueeze(torch.ones(4, 4),0), torch.unsqueeze(torch.ones(4, 4),0)))
# print(torch.ones(4, 4)*torch.ones(4, 4))
valid_encoder_pos_matrix = torch.bmm(vaild_encoder_pos, vaild_encoder_pos.transpose(1, 2))
invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
print(mask_encoder_self_attention)
#假设是vk注意力权重
score = torch.randn(batch_size, max(src_len), max(src_len))
masked_score = score.masked_fill(mask_encoder_self_attention, -np.inf)
prob = F.softmax(masked_score, -1)
print(prob)

torch.Size([2, 4, 1]) 
 tensor([[[1.],
         [1.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.]]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])
tensor([[[0.4808, 0.5192, 0.0000, 0.0000],
         [0.9586, 0.0414, 0.0000, 0.0000],
         [   nan,    nan,    nan,    nan],
         [   nan,    nan,    nan,    nan]],

        [[0.0808, 0.1797, 0.6634, 0.0761],
         [0.0682, 0.5986, 0.2450, 0.0882],
         [0.0549, 0.0838, 0.1006, 0.7608],
         [0.7652, 0.1342, 0.0709, 0.0297]]])


> 5、get_attn_pad_mask

In [621]:
enc_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 1)
dec_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(tgt_len)-L)),0) for L in tgt_len]), 2)
print(enc_encoder_pos.shape, "\n", dec_encoder_pos.shape)
print(enc_encoder_pos, "\n", dec_encoder_pos)
# 具体维度通过【】的奇偶特征决定
get_attn_pad_mask = torch.bmm(dec_encoder_pos, enc_encoder_pos, )
print(get_attn_pad_mask, "\n", get_attn_pad_mask.shape)
score = torch.randn(2, max(src_len), max(tgt_len))
masked_score = score.masked_fill((1-get_attn_pad_mask).to(torch.bool), -np.inf)
prob = F.softmax(masked_score, -1)
print(prob)

torch.Size([2, 1, 4]) 
 torch.Size([2, 4, 1])
tensor([[[1., 1., 0., 0.]],

        [[1., 1., 1., 1.]]]) 
 tensor([[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [0.]]])
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [0., 0., 0., 0.]]]) 
 torch.Size([2, 4, 4])
tensor([[[0.1897, 0.8103, 0.0000, 0.0000],
         [0.3828, 0.6172, 0.0000, 0.0000],
         [0.1205, 0.8795, 0.0000, 0.0000],
         [0.7199, 0.2801, 0.0000, 0.0000]],

        [[0.2825, 0.4079, 0.1964, 0.1133],
         [0.4752, 0.0532, 0.2524, 0.2193],
         [0.1081, 0.4313, 0.3477, 0.1129],
         [   nan,    nan,    nan,    nan]]])


> 6、dec_self_attn_mask

In [622]:
tril = torch.unsqueeze(torch.tril(torch.ones(max(tgt_len), max(tgt_len))), 0)
print(tril.shape)
# tril = tril.expand(2, 4, 4) 
tril = torch.cat((tril, tril ))
print(tril)
dec_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(tgt_len)-L)),0) for L in tgt_len]), 2)
valid_decoder_pos_matrix = torch.bmm(dec_encoder_pos,dec_encoder_pos.transpose(1, 2))
dec_self_attn_mask = valid_decoder_pos_matrix*tril
print(dec_self_attn_mask)

torch.Size([1, 4, 4])
tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]])
tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [0., 0., 0., 0.]]])


> 7、loss

In [623]:
logits = torch.randn(2,3,4)
# batchsize=2 seqlen=3 vocal_size=4
label = torch.randint(0, 4, (2,3))
logits = logits.transpose(1,2)